In [109]:
import numpy as np 
from qiskit import QuantumRegister, ClassicalRegister,QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator

In [90]:
def num_to_binary(n,N):
    # n is the number to be represented in binary
    # N is the number of digits of the rapresentation
    #es: num_to_binary(5,4) = 0101
    Nbits=2**N
    if n>=Nbits: return 0
    return bin(n+2*Nbits)[4:]

def HammingCircuit(N):
    #Returns a circuit with just the qbits labeled as the parity and signal
    registers=[]
    for i in range(2**N):
        prefix='s' #s stands for signal
        if i==0 or np.log2(i)==int(np.log2(i)): prefix='p' #c stands for parity
        registers.append(QuantumRegister(1,prefix+num_to_binary(i, N)))
    registers.append(ClassicalRegister(2**N+N))
    circuit=QuantumCircuit(*registers,QuantumRegister(N,'anc')) #circuit already with ancillas
    #circit=QuantumCircuit(*registers)
    circuit.N=N
    return circuit


Initializing Ancillas

In [118]:
circ=HammingCircuit(3)
nqubits=2**circ.N
for i in range(1,nqubits):
    for j in range(0,circ.N):
        if i & 2**j == 2**j: circ.cx(i,nqubits+j)

circ.draw()

p000_0: ────────────────────────────────────────────────────────────
                                                                    
p001_0: ──■─────────────────────────────────────────────────────────
          │                                                         
p010_0: ──┼────■────────────────────────────────────────────────────
          │    │                                                    
s011_0: ──┼────┼─────────■────■─────────────────────────────────────
          │    │         │    │                                     
p100_0: ──┼────┼────■────┼────┼─────────────────────────────────────
          │    │    │    │    │                                     
s101_0: ──┼────┼────┼────┼────┼────■────■───────────────────────────
          │    │    │    │    │    │    │                           
s110_0: ──┼────┼────┼────┼────┼────┼────┼────■─────────■────────────
          │    │    │    │    │    │    │    │         │            
s111_0: ──┼────┼────┼────┼────┼────┼────┼────┼────■────┼────■────■──
        ┌─┴─┐  │    │  ┌─┴─┐  │  ┌─┴─┐  │    │  ┌─┴─┐  │    │    │  
 anc_0: ┤ X ├──┼────┼──┤ X ├──┼──┤ X ├──┼────┼──┤ X ├──┼────┼────┼──
        └───┘┌─┴─┐  │  └───┘┌─┴─┐└───┘  │  ┌─┴─┐└───┘  │  ┌─┴─┐  │  
 anc_1: ─────┤ X ├──┼───────┤ X ├───────┼──┤ X ├───────┼──┤ X ├──┼──
             └───┘┌─┴─┐     └───┘     ┌─┴─┐└───┘     ┌─┴─┐└───┘┌─┴─┐
 anc_2: ──────────┤ X ├───────────────┤ X ├──────────┤ X ├─────┤ X ├
                  └───┘               └───┘          └───┘     └───┘
c48: 11/════════════════════════════════════════════════════════════

In [117]:
circ=HammingCircuit(3)
nqubits=2**circ.N
count=np.zeros(circ.N)
for i in range(1,nqubits):
    count=count+1
    for j in range(circ.N):
        count[j]=count[j]%(2**j)
        if count[j]==0 and i!=1: circ.x(nqubits+j)
        if i==1 and j!=0: circ.x(nqubits+j)
    circ.mct([*range(nqubits,nqubits+N)],i)
circ.draw()



p000_0: ──────────────────────────────────────────────────────────────────────
             ┌───┐                                                            
p001_0: ─────┤ X ├────────────────────────────────────────────────────────────
             └─┬─┘     ┌───┐                                                  
p010_0: ───────┼───────┤ X ├──────────────────────────────────────────────────
               │       └─┬─┘     ┌───┐                                        
s011_0: ───────┼─────────┼───────┤ X ├────────────────────────────────────────
               │         │       └─┬─┘     ┌───┐                              
p100_0: ───────┼─────────┼─────────┼───────┤ X ├──────────────────────────────
               │         │         │       └─┬─┘     ┌───┐                    
s101_0: ───────┼─────────┼─────────┼─────────┼───────┤ X ├────────────────────
               │         │         │         │       └─┬─┘     ┌───┐          
s110_0: ───────┼─────────┼─────────┼─────────┼─────────┼───────┤ X ├──────────
               │         │         │         │         │       └─┬─┘     ┌───┐
s111_0: ───────┼─────────┼─────────┼─────────┼─────────┼─────────┼───────┤ X ├
               │  ┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐└─┬─┘
 anc_0: ───────■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──
        ┌───┐  │  ├───┤  │  └───┘  │  ├───┤  │  └───┘  │  ├───┤  │  └───┘  │  
 anc_1: ┤ X ├──■──┤ X ├──■─────────■──┤ X ├──■─────────■──┤ X ├──■─────────■──
        ├───┤  │  └───┘  │         │  ├───┤  │         │  └───┘  │         │  
 anc_2: ┤ X ├──■─────────■─────────■──┤ X ├──■─────────■─────────■─────────■──
        └───┘                         └───┘                                   
c47: 11/══════════════════════════════════════════════════════════════════════